In [1]:
import sys
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install openpyxl

In [4]:
# import pandas as pd

# # Load the uploaded Excel file to examine its structure
# file_path = '../20230619_Gimpel_Pyrrhula_Pyrrhula_BP_AM.xlsx'
# xls = pd.ExcelFile(file_path)

# # Checking sheet names to understand the file's structure
# sheet_names = xls.sheet_names
# sheet_names


In [5]:
# portrait_df = pd.read_excel(xls, sheet_name='Pflanzenliste')
# portrait_df.head()

In [2]:
import pandas as pd
import os

# Verzeichnis, in dem sich alle Excel-Dateien befinden
directory = '../../species-portraits/portraits'

# Leeres DataFrame zum Sammeln der Daten
combined_df = pd.DataFrame()

# Alle Dateien im Verzeichnis durchgehen
for filename in os.listdir(directory):
    # Überprüfen, ob die Datei eine .xlsx-Datei ist
    if filename.endswith('.xlsx'):
        file_path = os.path.join(directory, filename)
        
        # Excel-Datei laden und das Sheet "Pflanzenliste" einlesen
        try:
            df = pd.read_excel(file_path, sheet_name='Pflanzenliste')
        except Exception as e:
            print(f"Fehler beim Laden der Datei {filename}: {e}")
            continue
        
        # Festlegen der möglichen Spaltennamen
        scientific_name_columns = ['Art_botanisch', 'Name_botanisch', 'scientific_name']
        common_name_columns = ['Art_deutsch', 'Name_deutsch', 'common_name']
        #optional_column = 'Kategorie'

        # Erforderliche Spalten finden
        scientific_name_col = next((col for col in scientific_name_columns if col in df.columns), None)
        common_name_col = next((col for col in common_name_columns if col in df.columns), None)

        # Prüfen, ob die erforderlichen Spalten vorhanden sind
        if scientific_name_col and common_name_col:
            df_selected = df[[scientific_name_col, common_name_col]].copy()

            # Wenn die optionale Spalte 'Kategorie' vorhanden ist, hinzufügen, sonst mit None füllen
            # if optional_column in df.columns:
            #     df_selected[optional_column] = df[optional_column]
            # else:
            #     df_selected[optional_column] = None
        else:
            print(f"Erforderliche Spalten fehlen in {filename}")
            print(f"Vorhandene Spalten: {df.columns}")
            continue
        
        # Umbenennen der Spalten in englische Bezeichnungen
        df_selected.columns = ['scientific_name', 'common_name']
        
        # Hinzufügen der Daten zum kombinierten DataFrame
        combined_df = pd.concat([combined_df, df_selected])

# Entfernen von Duplikaten
combined_df_unique = combined_df.drop_duplicates(subset=['scientific_name'])

# Hinzufügen einer 'id'-Spalte mit fortlaufenden Ganzzahlen
# combined_df_unique.insert(0, 'id', range(1, len(combined_df_unique) + 1))

combined_df_unique

Fehler beim Laden der Datei 20241011_Zwergfledermaus_PipistrellusPipistrellus_AM.xlsx: Worksheet named 'Pflanzenliste' not found
Fehler beim Laden der Datei 20241105_Zweifarbfledermaus_Vespertilio_murinus_CE_AM.xlsx: Worksheet named 'Pflanzenliste' not found
Fehler beim Laden der Datei 20230320_Grauschnäpper_BP_AM.xlsx: Worksheet named 'Pflanzenliste' not found
Fehler beim Laden der Datei 20240808_Zauneidechse_LacertaAgilis_AM.xlsx: Worksheet named 'Pflanzenliste' not found
Fehler beim Laden der Datei 20240715_Grünspecht_PicusViridis_AM.xlsx: Worksheet named 'Pflanzenliste' not found
Fehler beim Laden der Datei 20240709_Mauersegler_ApusApus_AM.xlsx: Worksheet named 'Pflanzenliste' not found


,scientific_name,common_name
0,Acer campestre,Feld-Ahorn
1,Acer pseudoplatanus,Berg-Ahorn
2,Aegopodium podagraria,Giersch
3,Antrhiscus sylvestris,Wiesen-Kerbel
4,Brassica napus,Raps
...,...,...
46,Symphytum tuberosum,Knoten-Beinwell
48,Tulipa gesneriana,Garten-Tulpe
49,Tulipa tarda,Tarda-Tulpe
51,Veronica teucrium,Großer Ehrenpreis


In [4]:
# add extra data from 'Pflanzenliste_Pflanzentyp'

plants_file_path = '../20241106_Pflanzenliste_Pflanzentyp_Datenbank.xlsx'
plants_xls = pd.ExcelFile(plants_file_path)

# # Checking sheet names to understand the file's structure
sheet_names = plants_xls.sheet_names

plants_df = pd.read_excel(plants_xls, sheet_name=sheet_names[0])
plants_df.head()



,Name_wiss,Name_dt,Pflanzentyp,Wuchshöhe (m),Blühzeit,Heimisch,Ökologische Bedeutung für heimische Fauna
0,Acer campestre,Feld-Ahorn,Gehölz,5 - 15,Mai/Juni,ja,Pollen- und Nektarquelle für Honig- und Wildbi...
1,Acer pseudoplatanus,Berg-Ahorn,Gehölz,30 - 40,Mai/Juni,ja,Nektar- und Pollenquelle für Honig- und Wildbi...
2,Aegopodium podagraria,Giersch,Staude,"0,3 - 0,8",Mai-August,ja,Pollen- und Nektarquelle für Honig- und Wildbi...
3,Aesculus hippocastanum,Gewöhnliche Rosskastanie,Gehölz,25 - 30,Mai/Juni,nein,Nektar- und Pollenquelle für Honig- und Wildbi...
4,Ajuga reptans,Kriechender Günsel,Staude,"0,1 - 0,3",Mai-August,ja,Pollen- und Nektarquelle für Honig- und Wildbi...


In [5]:
# Clean the column names for snake_case and English translation
plants_df.columns = [
    'scientific_name', 'common_name', 'plant_type', 'growing_height',
    'flowering_time', 'is_native', 'local_fauna_importance'
]

plants_df.head()

# enrich existing entries from combined_df_unique and add extra entries from 'Pflanzenliste_Pflanzentyp' to combined_df_unique
enriched_df = pd.merge(combined_df_unique, plants_df, on='scientific_name', how='outer', suffixes=('_combined', '_plants'))

# Create a new column for common_name that takes the value from plants_df where available, otherwise from combined_df_unique
enriched_df['common_name'] = enriched_df['common_name_plants'].combine_first(enriched_df['common_name_combined'])

# Drop the extra common_name columns
enriched_df = enriched_df.drop(columns=['common_name_plants', 'common_name_combined'])

enriched_df = enriched_df[['scientific_name', 'common_name', 'plant_type', 'flowering_time', 'is_native', 'local_fauna_importance']]

# Remove duplicate rows based on scientific_name
enriched_df = enriched_df.drop_duplicates(subset='scientific_name', keep='first')

# Drop rows where common_name is empty (NaN)
enriched_df = enriched_df.dropna(subset=['common_name'])

# Hinzufügen einer 'id'-Spalte mit fortlaufenden Ganzzahlen
enriched_df.insert(2, 'id', range(1, len(enriched_df) + 1))

enriched_df

,scientific_name,common_name,id,plant_type,flowering_time,is_native,local_fauna_importance
0,Acer campestre,Feld-Ahorn,1,Gehölz,Mai/Juni,ja,Pollen- und Nektarquelle für Honig- und Wildbi...
1,Acer campestris,Ahorn,2,NaN,NaN,NaN,NaN
2,Acer pseudoplatanus,Berg-Ahorn,3,Gehölz,Mai/Juni,ja,Nektar- und Pollenquelle für Honig- und Wildbi...
3,Acer sp.,Ahorn,4,NaN,NaN,NaN,NaN
4,Aegopodium podagraria,Giersch,5,Staude,Mai-August,ja,Pollen- und Nektarquelle für Honig- und Wildbi...
...,...,...,...,...,...,...,...
257,Vinca sp.,Immergrün,258,NaN,NaN,NaN,NaN
258,Viola arvensis,Ackerveilchen,259,Einjährige,April-Oktober,ja,Pollen- und Nektarquelle für Honigbienen; Raup...
259,Viola riviniana,Hain-Veilchen,260,Staude,April-Juli,ja,Pollen- und Nektarquelle für Honigbienen; Nekt...
260,Viola sp.,Veilchen,261,NaN,NaN,NaN,NaN


In [8]:
# Exportieren der bereinigten Daten als CSV-Datei
output_path = './out/plants/all_plants.csv'
enriched_df.to_csv(output_path, index=False)

print(f'Die Datei wurde erfolgreich exportiert nach: {output_path}')

Die Datei wurde erfolgreich exportiert nach: ./out/plants/all_plants.csv
